# Лабораторная работа: Перцептрон на синтетике

## 1. Подготовка данных

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
rng = 42
X, y = make_blobs(n_samples=500, centers=[(-2,-2),(2,2)],
                  cluster_std=1.2, n_features=2, random_state=rng)
y = np.where(y==0, -1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=rng, stratify=y)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap="bwr", alpha=0.7)
plt.title("Синтетические данные (train)")
plt.xlabel("x1"); plt.ylabel("x2"); plt.show()

## 2. Реализация перцептрона

In [ ]:
from typing import Tuple, List

def perceptron_train(X: np.ndarray, y: np.ndarray,
                     eta: float = 0.1, epochs: int = 50,
                     shuffle: bool = True, margin: float = 0.0,
                     seed: int = 0) -> Tuple[np.ndarray, float, List[int]]:
    rs = np.random.RandomState(seed)
    n, d = X.shape
    w = np.zeros(d)
    b = 0.0
    errors_hist = []

    for _ in range(epochs):
        idx = rs.permutation(n) if shuffle else np.arange(n)
        errors = 0
        for i in idx:
            score = np.dot(w, X[i]) + b
            if y[i] * score <= margin:
                w += eta * y[i] * X[i]
                b += eta * y[i]
                errors += 1
        errors_hist.append(errors)
    return w, b, errors_hist

def predict(X: np.ndarray, w: np.ndarray, b: float) -> np.ndarray:
    return np.where(X @ w + b >= 0, 1, -1)

## 3. Обучение и базовая оценка

In [ ]:
w, b, errs = perceptron_train(X_train, y_train, eta=0.1, epochs=30, margin=0.0, shuffle=True, seed=1)

y_pred = predict(X_test, w, b)
acc = (y_pred == y_test).mean()
print(f"Test accuracy: {acc:.3f}")

plt.plot(errs, marker="o")
plt.xlabel("Эпоха"); plt.ylabel("Ошибок за эпоху")
plt.title("Кривая обучения (число обновлений)")
plt.grid(True); plt.show()

## 4. Визуализация границы решений

In [ ]:
def plot_decision_boundary(X, y, w, b, title="Граница решений"):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 400),
                         np.linspace(y_min, y_max, 400))
    Z = np.sign(w[0]*xx + w[1]*yy + b)
    plt.contourf(xx, yy, Z, alpha=0.15, levels=[-1,0,1], cmap="bwr")
    plt.contour(xx, yy, w[0]*xx + w[1]*yy + b, levels=[0], linewidths=2)
    plt.scatter(X[:,0], X[:,1], c=y, cmap="bwr", edgecolors="k", alpha=0.8)
    plt.title(title); plt.xlabel("x1"); plt.ylabel("x2"); plt.show()

plot_decision_boundary(X_train, y_train, w, b, "Перцептрон: граница на train")
plot_decision_boundary(X_test,  y_test,  w, b, "Перцептрон: граница на test")

## 5. Исследовательская часть

5.1 Влияние шага обучения

In [ ]:
for eta in [0.01, 0.05, 0.1, 0.5]:
    w, b, errs = perceptron_train(X_train, y_train, eta=eta, epochs=30, seed=1)
    acc = (predict(X_test, w, b) == y_test).mean()
    print(f"eta={eta}: acc={acc:.3f}")
    plt.plot(errs, label=f"eta={eta}")
plt.legend(); plt.title("Ошибки при разных eta"); plt.show()

5.2 Перемешивание

In [ ]:
for shuffle in [True, False]:
    w, b, errs = perceptron_train(X_train, y_train, eta=0.1, epochs=30, shuffle=shuffle, seed=1)
    acc = (predict(X_test, w, b) == y_test).mean()
    print(f"shuffle={shuffle}: acc={acc:.3f}")
    plt.plot(errs, label=f"shuffle={shuffle}")
plt.legend(); plt.title("Ошибки при shuffle vs no-shuffle"); plt.show()

Сравнивались варианты shuffle=True и shuffle=False.

Результаты идентичные: 97.3%. 

Вывод: перемешивание не влияет на простых данных.

5.3 Маржа

In [ ]:
for margin in [0.0, 0.1, 0.5]:
    w, b, errs = perceptron_train(X_train, y_train, eta=0.1, epochs=30, margin=margin, seed=1)
    acc = (predict(X_test, w, b) == y_test).mean()
    print(f"margin={margin}: acc={acc:.3f}")
    plot_decision_boundary(X_train, y_train, w, b, f"margin={margin}")

Тестировались margin = {0.0, 0.1, 0.5}.

margin=0.0 → acc=97.3%

margin=0.1 → acc=99.3%

margin=0.5 → acc=98.7%

Вывод: добавление маржи может повысить точность и устойчивость обучения.

5.4 Неразделимые данные

In [ ]:
X2, y2 = make_blobs(n_samples=500, centers=[(-2,-2),(2,2)],
                    cluster_std=2.0, random_state=42)
y2 = np.where(y2==0, -1, 1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42, stratify=y2)

scaler2 = StandardScaler().fit(X2_train)
X2_train = scaler2.transform(X2_train)
X2_test = scaler2.transform(X2_test)

# ранняя остановка
patience = 5
w, b = np.zeros(2), 0.0
errs, best_errs = [], []
no_improve = 0
for epoch in range(50):
    w, b, err = perceptron_train(X2_train, y2_train, eta=0.1, epochs=1, seed=epoch)
    errors = err[-1]
    errs.append(errors)
    if epoch>0 and errs[-1]>=errs[-2]:
        no_improve+=1
    else:
        no_improve=0
    if no_improve>=patience:
        print(f"Early stopping at epoch {epoch}")
        break

plt.plot(errs); plt.title("Ошибки на неразделимых данных"); plt.show()

Были сгенерированы данные с сильным перекрытием классов. Введена стратегия ранней остановки.

Результат: обучение остановлено на 34-й эпохе.

Вывод: перцептрон не может идеально разделять такие данные, требуется регуляризация или более сложные модели.

5.5 Сравнение со стандартными методами

In [ ]:
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC

sk_perc = Perceptron(alpha=0.0, eta0=0.1, max_iter=1000, tol=1e-3, random_state=0)
sk_perc.fit(X_train, y_train)
print("sklearn Perceptron acc:", (sk_perc.predict(X_test)==y_test).mean())

logr = LogisticRegression().fit(X_train, y_train)
lsvc = LinearSVC().fit(X_train, y_train)
print("LogReg acc:", (logr.predict(X_test)==y_test).mean())
print("LinearSVC acc:", (lsvc.predict(X_test)==y_test).mean())

Perceptron (sklearn): 97.3%

Logistic Regression: 98.7%

Linear SVC: 98.7%

Вывод: классический перцептрон уступает современным методам, но показывает достойный результат.

## 6. Выводы

Перцептрон успешно обучается на синтетических разделимых данных, достигая точности ~97–99%.

Влияние шага обучения и перемешивания данных минимально для простых задач.

Использование маржи позволяет повысить устойчивость и точность модели.

На неразделимых данных необходимы более сложные методы (SVM, логистическая регрессия).

Сравнение с библиотечными реализациями подтвердило, что перцептрон — это базовая, но полезная модель для понимания принципов линейной классификации.